<a href="https://colab.research.google.com/github/awsdevguru/PearsonMLFoundations/blob/main/2_3_04_Encoding_Categorical_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encoding Categorical Data

## 1. Objectives

* Identify different types of categorical variables
* Apply multiple encoding techniques
* Compare how encoding affects model performance
* Learn to handle unseen categories safely in pipelines

## 2. Setup


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

df = None

## 3. Create Sample Dataset

Use a small dataset combining nominal, ordinal, and binary columns.

In [ ]:
def reset_data():
  global df
  data = {
    'Color': ['Red','Blue','Green','Blue','Red','Green','Blue','Red'],
    'Size': ['Small','Medium','Large','Medium','Small','Large','Large','Medium'],
    'Purchased': ['Yes','No','Yes','No','Yes','Yes','No','No']
  }
  df = pd.DataFrame(data)

reset_data()
df

## Why encoding is needed

Most machine learning models (like those in scikit-learn) work only with numerical data, they perform mathematical operations such as computing distances, sums, and dot products.

Words like "Red" or "Medium" can't be directly multiplied or compared numerically.

Therefore, we encode categorical features into numbers so that algorithms can use them.

**The following code block will fail.**



In [ ]:
# This will fail, can't predict categorical strings

X = df[['Color','Size']]
y = df['Purchased']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

## 4. Identify Variable Types

* **Nominal:** Color
* **Ordinal:** Size (Small < Medium < Large)
* **Binary:** Purchased

In [ ]:
df.info()

## 5. One-Hot Encoding (Nominal)

Each color becomes a binary column, good for linear models but expands feature space.

https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

In [ ]:
pd.get_dummies(df, columns=['Color'], drop_first=False)


## 6. Ordinal Encoding (Ordered)
Keeps order; ideal for tree models or ordinal data.

In [ ]:
size_order = [['Small','Medium','Large']]
ord_enc = OrdinalEncoder(categories=size_order)
df['Size_encoded'] = ord_enc.fit_transform(df[['Size']])
df

## 7. Label Encoding (Binary/Target)
Converts Yes/No to 1/0: compact representation.

In [ ]:
le = LabelEncoder()
df['Purchased_encoded'] = le.fit_transform(df['Purchased'])
df

## 8. Compare Impact on Model

Quick logistic regression demo to show encoding influence:

In [ ]:
X = pd.get_dummies(df[['Color','Size_encoded']], drop_first=True)
y = df['Purchased_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

## 9. Key Takeaways

* Choose encoding based on variable type and model type.
* One-hot: best for nominal & linear models.
* Ordinal: best for ordered categories.
* Label: for binary or tree models.
* Always plan for unseen categories in production.

# Visualizing Categories and Encodings

**A) Category Distributions (why one-hot explodes columns)**

Bar charts: more categories, more one-hot columns and sparsity.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10, 4))
df['Color'].value_counts().plot(kind='bar', ax=ax[0], title='Color Counts')
df['Size'].value_counts().plot(kind='bar', ax=ax[1], title='Size Counts (Ordinal)')
plt.tight_layout()
plt.show()

**B) Ordinal Mapping Check (order preserved)**

Scatter: ordinal encoding encodes rank, not distance accuracy.

In [ ]:
# Visual check: Size → Size_encoded relationship
plt.figure(figsize=(5,3))
plt.scatter(range(len(df)), df['Size_encoded'])
plt.yticks([0,1,2], ['Small','Medium','Large'])
plt.title('Ordinal Encoding Preserves Order')
plt.xlabel('Row')
plt.ylabel('Size (encoded)')
plt.show()

**C) One-Hot Matrix Heatmap (sparsity intuition)**

Heatmap: one-hot creates binary flags; most entries are zeros.

In [ ]:
# Build one-hot only for Color to keep it small
color_ohe = pd.get_dummies(df[['Color']], drop_first=False)
plt.figure(figsize=(5,3))
plt.imshow(color_ohe.values, aspect='auto', interpolation='nearest')
plt.title('One-Hot (Color): Sparse Binary Matrix')
plt.xlabel('Color dummy columns')
plt.ylabel('Rows')
plt.show()

**D) Encoding -> Coefficients (how models "see" categories)**

Coefficients: linear models assign separate weights per category dummy.

In [ ]:
# Minimal demo: one-hot Color + ordinal Size → logistic regression
X = pd.concat([pd.get_dummies(df['Color'], drop_first=True), df[['Size_encoded']]], axis=1)
y = df['Purchased_encoded']

from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(X, y)

coef = pd.Series(model.coef_.ravel(), index=X.columns).sort_values()
coef.plot(kind='barh', figsize=(6,4), title='Feature Coefficients')
plt.tight_layout()
plt.show()